## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Cherskiy,RU,68.75,161.30,4.33,94,100,5.26,overcast clouds
1,1,Changping,CN,40.22,116.22,53.01,33,0,4.47,clear sky
2,2,Vaini,TO,-21.20,-175.20,75.20,88,75,8.05,broken clouds
3,3,Sibiti,CG,-3.68,13.35,69.73,99,100,3.33,overcast clouds
4,4,Dikson,RU,73.51,80.55,19.63,98,96,9.06,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Vaini,TO,-21.20,-175.20,75.20,88,75,8.05,broken clouds
5,5,Butaritari,KI,3.07,172.79,81.79,78,1,10.60,clear sky
6,6,Carpinteria,US,34.40,-119.52,84.00,59,69,3.60,broken clouds
10,10,Gat,IL,31.61,34.76,75.00,68,100,3.69,overcast clouds
22,22,Rikitea,PF,-23.12,-134.97,74.55,86,77,22.30,light rain
23,23,Cayenne,GF,4.93,-52.33,82.40,78,69,8.05,light rain
39,39,Isiolo,KE,0.35,37.58,71.33,63,48,13.09,scattered clouds
42,42,Kavaratti,IN,10.57,72.64,81.88,71,0,4.29,clear sky
43,43,Kilakarai,IN,9.23,78.78,80.51,80,40,8.10,moderate rain
47,47,Gasa,PS,31.50,34.47,75.20,69,20,5.82,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                217
City                   217
Country                215
Lat                    217
Lng                    217
Max Temp               217
Humidity               217
Cloudiness             217
Wind Speed             217
Current Description    217
dtype: int64

In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Vaini,TO,-21.20,-175.20,75.20,88,75,8.05,broken clouds
5,5,Butaritari,KI,3.07,172.79,81.79,78,1,10.60,clear sky
6,6,Carpinteria,US,34.40,-119.52,84.00,59,69,3.60,broken clouds
10,10,Gat,IL,31.61,34.76,75.00,68,100,3.69,overcast clouds
22,22,Rikitea,PF,-23.12,-134.97,74.55,86,77,22.30,light rain
...,...,...,...,...,...,...,...,...,...,...
664,664,Muheza,TZ,-5.17,38.78,71.13,92,28,1.88,light rain
667,667,Puerto Del Rosario,ES,28.50,-13.86,71.60,83,20,18.34,few clouds
670,670,Marakkanam,IN,12.20,79.95,79.86,77,54,4.52,broken clouds
675,675,Gonbad-E Qabus,IR,37.25,55.17,72.32,28,96,5.68,overcast clouds


In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Vaini,TO,75.20,broken clouds,-21.20,-175.20,
5,Butaritari,KI,81.79,clear sky,3.07,172.79,
6,Carpinteria,US,84.00,broken clouds,34.40,-119.52,
10,Gat,IL,75.00,overcast clouds,31.61,34.76,
22,Rikitea,PF,74.55,light rain,-23.12,-134.97,
23,Cayenne,GF,82.40,light rain,4.93,-52.33,
39,Isiolo,KE,71.33,scattered clouds,0.35,37.58,
42,Kavaratti,IN,81.88,clear sky,10.57,72.64,
43,Kilakarai,IN,80.51,moderate rain,9.23,78.78,
47,Gasa,PS,75.20,few clouds,31.50,34.47,


In [12]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

hotel_df.head(10)

# 6b. Iterate through the hotel DataFrame.

    # 6c. Get latitude and longitude from DataFrame

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    

    # 6e. Make request and retrieve the JSON data from the search. 
   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Vaini,TO,75.20,broken clouds,-21.20,-175.20,Keleti Beach Resort
5,Butaritari,KI,81.79,clear sky,3.07,172.79,Isles Sunset Lodge
6,Carpinteria,US,84.00,broken clouds,34.40,-119.52,Holiday Inn Express & Suites Carpinteria
10,Gat,IL,75.00,overcast clouds,31.61,34.76,OlusHome
22,Rikitea,PF,74.55,light rain,-23.12,-134.97,Pension Maro'i
23,Cayenne,GF,82.40,light rain,4.93,-52.33,Hôtel Le Dronmi
39,Isiolo,KE,71.33,scattered clouds,0.35,37.58,Gaddisa Hotel
42,Kavaratti,IN,81.88,clear sky,10.57,72.64,Hotels in Lakshadweep Islands
43,Kilakarai,IN,80.51,moderate rain,9.23,78.78,Musthafa Maraikayar Garden
47,Gasa,PS,75.20,few clouds,31.50,34.47,رابطة الفنانين الفلسطينيين


In [30]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(how="any")
#clean_hotel_df = hotel_df[hotel_df["Hotel Name"].notna()]
clean_hotel_df.head(10)
len(clean_hotel_df)


215

In [31]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [35]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [36]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))